# About this example

This example shows how you can deploy ALBERT model to analyze confidential text for sentiment analysis. The model will be left untrained for demonstration purposes. We could finetune it on positive/negative samples before deploying it.

By using BlindAI, people can send data for the AI to analyze sensitive text without having to fear privacy leaks.

ALBERT is a state of the art Transformers model for NLP. You can learn more about it on this [Hugging Face page](https://huggingface.co/docs/transformers/model_doc/albert).

More information on this use case can be found on our blog post [Deploy Transformers models with confidentiality](https://blog.mithrilsecurity.io/transformers-with-confidentiality/).

# Installing dependencies

Install the dependencies this example needs.

In [1]:
!pip install transformers[onnx] torch

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.utils import int_from_bytes


Install the latest version of BlindAI.

In [3]:
!pip install blindai

# Preparing the model

In this first step we will export a standard Hugging Face ALBERT model to an ONNX file, as BlindAI accepts only ONNX files.

In [103]:
import os
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_name = "albert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

text = "Paris is the [MASK] of France."
tokenizer_output = tokenizer(text, return_tensors="pt")

input_ids = tokenizer_output["input_ids"]
attention_mask = tokenizer_output["attention_mask"]
token_type_ids = tokenizer_output["token_type_ids"]

dynamic_axes = {
    0: "batch",
    1: "seq",
}

output_dir = "./albert"
os.makedirs(output_dir, exist_ok=True)
torch.onnx.export(
    model,
    (input_ids, attention_mask, token_type_ids),
    os.path.join(output_dir, "model.onnx"),
    input_names=["input_ids", "attention_mask", "token_type_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": dynamic_axes,
        "attention_mask": dynamic_axes,
        "token_type_ids": dynamic_axes,
        "logits": dynamic_axes,
    },
    opset_version=13,
)

tokenizer.save_pretrained(output_dir)

('./albert/tokenizer_config.json',
 './albert/special_tokens_map.json',
 './albert/tokenizer.json')

# Deployment on BlindAI

Please make sure the **server is running**. To launch the server, refer to the [Launching the server](https://docs.mithrilsecurity.io/getting-started/quick-start/run-the-blindai-server) documentation page. 

If you have followed the steps and have the Docker image ready, this mean you simply have to run `docker run -it -p 50051:50051 -p 50052:50052 mithrilsecuritysas/blindai-server-sim:latest`

So the first thing we need to do is to connect securely to the BlindAI server instance. Here we will use simulation mode for ease of use. This means that we do not leverage the hardware security propertiers of secure enclaves, but we do not need to run the Docker image with a specific hardware.

If you wish to run this example in hardware mode, you need to prepare the `host_server.pem` and `policy.toml` files. Learn more on the [Deploy on Hardware](https://docs.mithrilsecurity.io/getting-started/deploy-on-hardware) documentation page. 

In [102]:
from blindai.client import BlindAiClient, ModelDatumType

# Launch client
client = BlindAiClient()

# Simulation mode
client.connect_server(addr="localhost", simulation=True)

# Hardware mode
# client.connect_server(addr="localhost", policy="./policy.toml", certificate="./host_server.pem")

Then, upload the model inside the BlindAI server. This simply means uploading the ONNX file created before.

When uploading the model, we have to precise the shape of the input and the data type. In this case, because we use Transformers model with tokens, we simply need to send the indices of the tokens, i.e. integers.

And in the case of a multiple input model like ALBERT, we have to provide each input and its datum type in a list just as provided below.

In [104]:
tensor_inputs = [
    [input_ids.shape, ModelDatumType.I64],
    [token_type_ids.shape, ModelDatumType.I64],
    [attention_mask.shape, ModelDatumType.I64]
]

tensor_outputs = ModelDatumType.F32
client.upload_model(model="./albert/model.onnx", tensor_inputs=tensor_inputs, tensor_outputs=tensor_outputs)

# Sending data for confidential prediction

Now it's time to check it's working live!

We will just prepare the inputs for the model inside the secure enclave of BlindAI to process it.

In [105]:
inputs = tokenizer(text)["input_ids"]
token_type = tokenizer(text)["token_type_ids"]
attention = tokenizer(text)["attention_mask"]

Now we can get a prediction.

In [132]:
response = client.run_model([inputs, token_type, attention])
print(response.output[:20]) # Reducing the output if it's very long

[6.296055793762207, 1.568918228149414, 2.0557966232299805, 4.909221172332764, 7.076786518096924, -1.025282382965088, 4.391934394836426, -2.521080255508423, 0.37863948941230774, 3.25677227973938, 6.41582727432251, 2.833071231842041, -7.473093509674072, -2.3739407062530518, 8.971452713012695, 3.583134174346924, 7.851095676422119, 4.262254238128662, 2.3475804328918457, 9.010588645935059]


Here we can compare the results against the original prediction.

In [136]:

outputs = model(torch.tensor(inputs).unsqueeze(0), torch.tensor(
    token_type).unsqueeze(0), torch.tensor(attention).unsqueeze(0)).logits.detach()

print(outputs[0][0].tolist()[:20])


[6.294389724731445, 1.5712192058563232, 2.0538954734802246, 4.907917499542236, 7.075641632080078, -1.021797776222229, 4.391971588134766, -2.5168683528900146, 0.3809121549129486, 3.2570388317108154, 6.416499137878418, 2.8358120918273926, -7.472112655639648, -2.371373414993286, 8.972518920898438, 3.582233428955078, 7.852456092834473, 4.262726783752441, 2.347175359725952, 9.01127815246582]


Et voila! We have been able to apply a start of the art model of NLP, without ever having to show the data in clear to the people operating the service!

If you have liked this example, do not hesitate to drop a star on our [GitHub](https://github.com/mithril-security/blindai) and chat with us on our [Discord](https://discord.gg/TxEHagpWd4)!